# Math Parser
This solution uses a full-blown parser to evaluate the math expressions. I'm using a recursive descent parser to build an
Abstract Syntax Tree (AST) representing the mathematical expression and then using a `Visitor` to simplify that AST into
a final `MathConstant` containing the result.

In [46]:
class MathExpression(object):
    def get_value(self) -> int:
        raise NotImplementedError()

class MathConstant(MathExpression):
    def __init__(self, value: int):
        self.value = value

    def __eq__(self, other):
        return isinstance(other, MathConstant) and self.value == other.value

    def __str__(self):
        return str(self.value)

    def __repr__(self):
        return f"MathConstant({self.value})"

class MathGroup(MathExpression):
    def __init__(self, child: MathExpression):
        self.child = child

    def __eq__(self, other):
        return isinstance(other, MathGroup) and self.child == other.child

    def __str__(self):
        return f"({str(self.child)})"

    def __repr__(self):
        return f"MathGroup({repr(self.child)})"

class MathBinaryExpression(MathExpression):
    def __init__(self, left: MathExpression, operator: str, right: MathExpression):
        self.left = left
        self.operator = operator
        self.right = right

    def __eq__(self, other):
        return isinstance(other, MathBinaryExpression) and self.left == other.left and self.operator == other.operator and self.right == other.right

    def __str__(self):
        return f"{str(self.left)} {self.operator} {str(self.right)}"

    def __repr__(self):
        return f"MathBinaryExpression({repr(self.left)}, '{self.operator}', {repr(self.right)})"

## Visitor
We're using a visitor pattern implementation to navigate the AST, which makes evaluating the final values relatively straightforward.
This also (concievably) allows us to do things like simplifying the tree, printing out step-by-step solutions etc.

In [84]:
class Visitor(object):
    def visit(self, expression: MathExpression) -> MathExpression:
        if isinstance(expression, MathConstant):
            return self.visit_constant(expression)

        if isinstance(expression, MathGroup):
            return self.visit_group(expression)

        if isinstance(expression, MathBinaryExpression):
            return self.visit_binary(expression)

    def visit_constant(self, expression: MathConstant) -> MathExpression:
        return expression

    def visit_group(self, expression: MathGroup) -> MathExpression:
        return MathGroup(self.visit(expression.child))

    def visit_binary(self, expression: MathBinaryExpression) -> MathExpression:
        return MathBinaryExpression(self.visit(expression.left), expression.operator, self.visit(expression.right))


class EvaluateVisitor(Visitor):
    def visit_group(self, expression: MathGroup):
        return self.visit(expression.child)

    def visit_binary(self, expression: MathBinaryExpression):
        operators = {
            '+': lambda a, b: a + b,
            '*': lambda a, b: a * b,
        }

        if expression.operator not in operators:
            raise NotImplementedError(f"The '{expression.operator}' operator is not currently supported.")

        left = self.visit(expression.left)
        right = self.visit(expression.right)

        assert isinstance(left, MathConstant)
        assert isinstance(right, MathConstant)

        return MathConstant(operators[expression.operator](left.value, right.value))

evaluator = EvaluateVisitor()


assert evaluator.visit(MathBinaryExpression(MathConstant(5), '+', MathBinaryExpression(MathConstant(3), '*', MathConstant(2)))) == MathConstant(11)


## Tokenizer

The tokenizer is a relatively simple `LR(1)` tokenizer implementation that generates a stream of distinct tokens
which will act as the input to the parser. Using a tokenizer means that the parser doesn't need to think about
where tokens start and end, simplifying some of the code and making it possible to index into specific tokens
positionally (i.e. `tokens[0]`, `tokens[1]` etc.).

To keep the token implementation easy to read, we're using a generator and storing this in a list. With a bit of
extra effort, we could retain only the current token in memory and generate the remainder on demand, allowing us
to parse potentially very large expressions efficiently.

In [ ]:
from typing import List, Iterator, Dict

class MathTokenizer(object):
    def parse(self, input: str) -> List[str]:
        return list(self.token_stream(input))

    def token_stream(self, input: str) -> Iterator[str]:
        position = 0
        while position < len(input):
            if input[position] in ['(', ')', '+', '*']:
                yield input[position]
                position += 1
            elif input[position].isdigit():
                digits = ""
                while position < len(input) and input[position].isdigit():
                    digits += input[position]
                    position += 1

                yield digits
            else:
                position += 1

tokenizer = MathTokenizer()
assert tokenizer.parse("1 + 12 * (3 + 30)") == ["1", "+", "12", "*", "(", "3", "+", "30", ")"]

## Parser
The parser is a Recursive Descent parser with Operator-Precedence support. A call to the `.parse(tokens)` method will provide you with a `MathExpression` which can then be consumed through the `Visitor`, or debugged directly.

In [96]:
from typing import List, Dict


class MathParser(object):
    def __init__(self, operator_precedence: Dict[str, int] = None):
        self.operator_precedence = operator_precedence or {
            "+": 0,
            "*": 0
        }

    def parse(self, tokens: List[str]) -> MathExpression:
        result = self.parse_binary(tokens)
        assert tokens == []

        return result

    def parse_binary(self, tokens: List[str], min_precedence: int = 0) -> MathBinaryExpression:
        left = self.parse_group_or_constant(tokens)

        while tokens and tokens[0] in self.operator_precedence and self.operator_precedence[tokens[0]] >= min_precedence:
            operator = tokens.pop(0)

            if self.operator_precedence[operator] == min_precedence:
                right = self.parse_group_or_constant(tokens)
            else:
                right = self.parse_binary(tokens, min_precedence)

                # We need to elevate the minimum precedence for this step so that we don't inadvertently merge lower precedence entries
                min_precedence = self.operator_precedence[operator]

            left = MathBinaryExpression(left, operator, right)

        return left

    def parse_group_or_constant(self, tokens: List[str]) -> MathExpression:
        if tokens[0] == "(":
            return self.parse_group(tokens)
        else:
            return self.parse_constant(tokens)

    def parse_group(self, tokens: List[str]) -> MathGroup:
        assert tokens.pop(0) == "("
        child = self.parse_binary(tokens)
        assert tokens.pop(0) == ")"
        return MathGroup(child)

    def parse_constant(self, tokens: List[str]) -> MathConstant:
        return MathConstant(int(tokens.pop(0)))


parser = MathParser()

test_cases = {
    "1 + (2 * 3) + (4 * (5 + 6))": 51,
    "2 * 3 + (4 * 5)": 26,
    "1 + 2 * 3 + 4 * 5 + 6": 71,
    "5 + (8 * 3 + 9 + 3 * 4 * 3)": 437,
    "5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))": 12240,
    "((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2": 13632
}

for round_trip, final_value in test_cases.items():
    tokens = tokenizer.parse(round_trip)
    parsed = parser.parse(tokens)
    print(f"Parsed [{round_trip}] into [{str(parsed)}]")
    assert str(parsed) == round_trip

    evaluated = evaluator.visit(parsed)

    print(f"  evaluate(parsed) -> {evaluated}")
    assert evaluated.value == final_value


Parsed [1 + (2 * 3) + (4 * (5 + 6))] into [1 + (2 * 3) + (4 * (5 + 6))]
  evaluate(parsed) -> 51
Parsed [2 * 3 + (4 * 5)] into [2 * 3 + (4 * 5)]
  evaluate(parsed) -> 26
Parsed [1 + 2 * 3 + 4 * 5 + 6] into [1 + 2 * 3 + 4 * 5 + 6]
  evaluate(parsed) -> 71
Parsed [5 + (8 * 3 + 9 + 3 * 4 * 3)] into [5 + (8 * 3 + 9 + 3 * 4 * 3)]
  evaluate(parsed) -> 437
Parsed [5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))] into [5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))]
  evaluate(parsed) -> 12240
Parsed [((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2] into [((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2]
  evaluate(parsed) -> 13632


In [95]:
parser2 = MathParser({
    "+": 0,
    "*": 1
})

test_cases = {
    "1 + 2 * 3 + 4 * 5 + 6": MathBinaryExpression(
        MathBinaryExpression(MathConstant(1), "+", MathConstant(2)),
        "*",
        MathBinaryExpression(
            MathBinaryExpression(MathConstant(3), "+", MathConstant(4)),
            "*",
            MathBinaryExpression(MathConstant(5), "+", MathConstant(6))
        ))
}
for expression, tree in test_cases.items():
    tokens = tokenizer.parse(expression)
    parsed = parser2.parse(tokens)
    print(f"Parsed [{repr(parsed)}]")
    print(f"   vs. [{repr(tree)}]")

    assert parsed == tree

test_cases = {
    "1 + 2 * 3 + 4 * 5 + 6": 231,
    "1 + (2 * 3) + (4 * (5 + 6))": 51,
    "2 * 3 + (4 * 5)": 46,
    "5 + (8 * 3 + 9 + 3 * 4 * 3)": 1445,
    "5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))": 669060,
    "((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2": 23340
}

for round_trip, final_value in test_cases.items():
    tokens = tokenizer.parse(round_trip)
    parsed = parser2.parse(tokens)
    print(f"Parsed [{round_trip}] into [{str(parsed)}]")
    assert str(parsed) == round_trip

    evaluated = evaluator.visit(parsed)

    print(f"  evaluate(parsed) -> {evaluated}")
    assert evaluated.value == final_value

Parsed [MathBinaryExpression(MathBinaryExpression(MathConstant(1), '+', MathConstant(2)), '*', MathBinaryExpression(MathBinaryExpression(MathConstant(3), '+', MathConstant(4)), '*', MathBinaryExpression(MathConstant(5), '+', MathConstant(6))))]
   vs. [MathBinaryExpression(MathBinaryExpression(MathConstant(1), '+', MathConstant(2)), '*', MathBinaryExpression(MathBinaryExpression(MathConstant(3), '+', MathConstant(4)), '*', MathBinaryExpression(MathConstant(5), '+', MathConstant(6))))]
Parsed [1 + 2 * 3 + 4 * 5 + 6] into [1 + 2 * 3 + 4 * 5 + 6]
  evaluate(parsed) -> 231
Parsed [1 + (2 * 3) + (4 * (5 + 6))] into [1 + (2 * 3) + (4 * (5 + 6))]
  evaluate(parsed) -> 51
Parsed [2 * 3 + (4 * 5)] into [2 * 3 + (4 * 5)]
  evaluate(parsed) -> 46
Parsed [5 + (8 * 3 + 9 + 3 * 4 * 3)] into [5 + (8 * 3 + 9 + 3 * 4 * 3)]
  evaluate(parsed) -> 1445
Parsed [5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))] into [5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))]
  evaluate(parsed) -> 669060
Parsed [((2 + 4 * 9) * (6 

In [89]:
with open('day18.txt', 'r') as f:
    expressions = f.readlines()

total_sum = 0
for expression in expressions:
    tokens = tokenizer.parse(expression)
    parsed = parser.parse(tokens)
    evaluated = evaluator.visit(parsed)
    total_sum += evaluated.value

print(f"Total Sum (part 1): {total_sum}")

Total Sum (part 1): 21022630974613


In [91]:
total_sum2 = 0
for expression in expressions:
    tokens = tokenizer.parse(expression)
    parsed = parser2.parse(tokens)
    evaluated = evaluator.visit(parsed)
    total_sum2 += evaluated.value

print(f"Total Sum (part 2): {total_sum2}")

Total Sum (part 2): 169899524778212
